# TensorFlow Time Series Predictions

Play with the same data as linear regressions. Based on:

- https://www.tensorflow.org/tutorials/structured_data/time_series

## Get Basic Setup Done

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import pandas as pd
import util
import datetime

mpl.rcParams['figure.figsize'] = (16, 8)
mpl.rcParams['axes.grid'] = False

# we may optionally define this further down, but set to None right here to allow
# it to be tested
tensorboard_callback = None

## Generate Test Data

In [ ]:
# Number of days to generate data for
DAYS = 10

# generate a numpy array of raw data first
d = util.gen_data(days=DAYS)

# turn it into a pandas data frame
df = pd.DataFrame({'Time': d[:, 0], 'ACEs': d[:, 1]})

# plot just the ACEs series
plt.plot(df['ACEs'])

## Univariate Data Extraction

This function takes in a 1D dataset of values (`dataset`) and carves it up into two different returns:

- An array of arrays of length `history_size` of overlapping data, starting with the data at `start_index`, ending at `end_index`.
- A simple array of future values that are `target_size` ticks in the future from each of the arrays above that are effectively the value we're trying to train to/for.

So, for setting `target_size`, consider how far into the future you want to predict, and for `history_size` consider how much of past history you want to consider.

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    '''
    * dataset the 1D array of data
    * start_index where in dataset to get data from
    * end_index last index to take data from
    * history_size size of past window of information
    * target_size how far in the future to predict
    '''
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [ ]:
uni_data = df['ACEs']
uni_data.index = df['Time']
uni_data.plot(subplots=True)

## Normalize Data

In [ ]:
uni_train_mean = uni_data.mean()
uni_train_std = uni_data.std()

print('Mean    = {}'.format(uni_train_mean))
print('Std Dev = {}'.format(uni_train_std))

# normalize all elements
uni_data = (uni_data-uni_train_mean)/uni_train_std
    
uni_data.plot(subplots=True)

## Prepare Data

In [ ]:
uni_data_vals = uni_data.values

# uses the past "day" for history (one tick is one minute)
# univariate_past_history = 1440

# try 30 minutes of history
univariate_past_history = 30

# predict 1 minute ahead
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data_vals, 0, 1440*(DAYS-2),
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data_vals, 1440*(DAYS-2), None,
                                       univariate_past_history,
                                       univariate_future_target)

In [ ]:
print(x_train_uni.shape)
print(x_train_uni)
print(y_train_uni.shape)
print(y_train_uni)

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target hardware resource utilization to predict (normalized)')
print (y_train_uni[0])

In [ ]:
def create_time_steps(length):
    return list(range(-length, 0))

In [ ]:
#
# Plot data, depending on its shape , and add in the "true future" as a green cross,
# and a prediction, if available, as a red dot
#
# The shape here is defined by the `univariate_past_history` defined earlier
#
def show_plot(plot_data, delta, title):
    labels = ['History', 'True Future', 'Model Prediction']
    marker = ['.-', 'gx', 'ro']
    time_steps = create_time_steps(plot_data[0].shape[0])
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future+5)*2])
    plt.xlabel('Time-Step')
    return plt

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example').show()

In [ ]:
show_plot([x_train_uni[500], y_train_uni[500]], 0, 'Sample Example').show()

## Take Average of Last Day of Observations & Predict

As best practice, always good to take a simple baseline prediction and check that whatever you do with respect to ML improves on that! In this case, take a simple everage of one of the training batches.

In [ ]:
def baseline(history):
    return np.mean(history)

In [ ]:
show_plot([x_train_uni[500], y_train_uni[500], baseline(x_train_uni[500])], 0, 'Baseline Prediction Example')

This is really not a very good prediction mechanism!!

## Train A Recurrent Neural Network

First, look at the basic shape of the data we're using for training:

In [ ]:
print(x_train_uni.shape)
print(y_train_uni.shape)

### Batch Up The Data Into TF Data Structures

Not going to go into this in detail, but first we need to split the training data in batches

In [ ]:
BATCH_SIZE = 256
BUFFER_SIZE = x_train_uni.shape[0]

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

### Compile The Model

First, define a function to create and compile the model. We use "LSTM" (Long Short-Term Memory") cells as time series predictions need to learn from past observartions to predict the next value in the sequence.

First we define a coiuple of functions to create models:

In [ ]:
BREADTH = 30

#
# create and compile a single-layer model
#
def create_model_single(breadth=BREADTH, input_shape=None):
    assert input_shape is not None
    retval = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(BREADTH, input_shape=input_shape),
        tf.keras.layers.Dense(1)
    ])
    retval.compile(optimizer='adam', loss='mae')
    return retval

#
# create and compile a multi-layer model
#
def create_model_multi(breadth=BREADTH, input_shape=None):
    assert input_shape is not None
    retval = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(BREADTH, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.LSTM(BREADTH, return_sequences=True),
        tf.keras.layers.LSTM(BREADTH),
        tf.keras.layers.Dense(1)
    ])
    retval.compile(optimizer='adam', loss='mae')
    return retval    

### Create Model Instances

In [ ]:
single_lstm_model = create_model_single(input_shape=x_train_uni.shape[-2:])
multi_lstm_model  = create_model_multi(input_shape=x_train_uni.shape[-2:])

Take a look at what a prediction will give you by taking one window from the validation set and calling `predict`:

In [ ]:
for x, y in val_univariate.take(1):
    print(single_lstm_model.predict(x).shape)
    print(multi_lstm_model.predict(x).shape)

### Model Training

Now we can train the model if we want. But skip this if you just want to load pre-created weights:

In [ ]:
EVALUATION_INTERVAL = 200  # original 200
VALIDATION_STEPS    = 50   # original 50
EPOCHS              = 10   # original 5

Define a function for logging data for TensorBoard:

In [ ]:
def tensorboard_callback(l=None, evaluations=None, epochs=None, validations=None):
    assert l is not None
    assert evaluations is not None
    assert epochs is not None
    assert validations is not None
    
    log_dir = "logs/fit/" + \
              "%s-%s-%03d-%03d-%03d" % (datetime.datetime.now().strftime("%m%d-%H%M"), l, epochs, evaluations, validations)
    return tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Train Single Layer Model

In [ ]:
history_lstm_single = single_lstm_model.fit(
    train_univariate,
    epochs=EPOCHS,
    steps_per_epoch=EVALUATION_INTERVAL,
    validation_data=val_univariate,
    validation_steps=VALIDATION_STEPS,
    callbacks=[tensorboard_callback(l='single', evaluations=EVALUATION_INTERVAL, epochs=EPOCHS, validations=VALIDATION_STEPS)])

#### Train Multi-Layer Model

In [ ]:
history_lstm_multi = multi_lstm_model.fit(
    train_univariate,
    epochs=EPOCHS,
    steps_per_epoch=EVALUATION_INTERVAL,
    validation_data=val_univariate,
    validation_steps=VALIDATION_STEPS,
    callbacks=[tensorboard_callback(l='multi', evaluations=EVALUATION_INTERVAL, epochs=EPOCHS, validations=VALIDATION_STEPS)])

#### Save Model Weights & Normalization  Factors For Reuse

Save the weights. Again, don't do this if we're loading a predefined set of weights. We also need to save the normalization factors we calculated over the training data.

In [ ]:
single_lstm_model.save_weights(
    'lstm_weights-%s-%03d-%03d-%03d' % ('single', EPOCHS, EVALUATION_INTERVAL, VALIDATION_STEPS))
multi_lstm_model.save_weights(
    'lstm_weights-%s-%03d-%03d-%03d' % ('multi', EPOCHS, EVALUATION_INTERVAL, VALIDATION_STEPS))

# save out the mean & stddev for use later in another notebook
with open('mean_stddev.json', 'w', encoding='utf-8') as f:
    data = {
        'mean': uni_train_mean,
        'stddev': uni_train_std
    }
    json.dump(data, f, ensure_ascii=False, indent=2)

### Load Models From Pre-Defined Weights Files

Load a predefined set of weights. Note that the model definition really needs top be the same, so look out for that, noting that some weights files will load into a model that is not the same; seems to happen when you have more layers in the model you trained than you are loading into:

In [ ]:
# generate a filename for weights based on these parameters
LOAD_EPOCHS = 30
LOAD_EVALUATION_INTERVAL = 100
LOAD_VALIDATION_STEPS = 25

In [ ]:
# single-layer model
loaded_single_lstm_model = create_model_single(input_shape=x_train_uni.shape[-2:])
loaded_single_lstm_model.load_weights('weights-prerun/lstm_weights-single-%03d-%03d-%03d' % (LOAD_EPOCHS, LOAD_EVALUATION_INTERVAL, LOAD_VALIDATION_STEPS))

In [ ]:
# multi-layer model
loaded_multi_lstm_model = create_model_multi(input_shape=x_train_uni.shape[-2:])
loaded_multi_lstm_model.load_weights('weights-prerun/lstm_weights-multi-%03d-%03d-%03d' % (LOAD_EPOCHS, LOAD_EVALUATION_INTERVAL, LOAD_VALIDATION_STEPS))

In [ ]:
loaded_single_lstm_model.summary()
loaded_multi_lstm_model.summary()

## Do Some Sample Predictions

This loop will display a plot per prediction.

In [ ]:
for x, y in val_univariate.take(25):
    plot = show_plot([x[0].numpy(), y[0].numpy(), loaded_single_lstm_model.predict(x)[0]], 0, 'Single-Layer LSTM model')
    plot.show()
    plot = show_plot([x[0].numpy(), y[0].numpy(), loaded_multi_lstm_model.predict(x)[0]], 0, 'Multi-Layer LSTM model')
    plot.show()

# Calculate Mean Squared Errors

Simnple MSE calculation against a random selection of the validation data created earlier.

In [ ]:
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# lstm_model = simple_lstm_model
lstm_model = loaded_lstm_model

real = []
predictions = []
for x, y in tqdm(val_univariate.take(250)):
    
    # print(x.shape)
    
    # predicted value, scaled back up
    predictions.append(lstm_model.predict(x)[0][0] * uni_train_std + uni_train_mean)
    
    # real value, scaled back up
    real.append(y[0].numpy() * uni_train_std + uni_train_mean)

error = mean_squared_error(real, predictions)
print('Test MSE : %.9f' % error)

# Scratch Area

In [ ]:
dataset = df
start_index = 0
end_index = 1440 * 3
history_size = 1440
target_size = 0

data = []
labels = []

start_index = start_index + history_size
if end_index is None:
    end_index = len(dataset) - target_size

for i in range(start_index, end_index):
    print('i', i)
    indices = range(i-history_size, i)
    print('indices', indices)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])

np.array(data), np.array(labels)

In [ ]:
dataset['Time']